In [1]:
import pandas as pd
import json
from pycarol import loginCarol, entityTemplateCarol, fieldsCarol


with open('../carol-ds-retail/config.json') as json_data:
    d = json.loads(json_data.read())
token_object = loginCarol.loginCarol(**d['mario'])
token_object.newToken()
print(token_object.access_token)

fd9fee10ceec11e7b2440242ac110003


In [2]:
token_object.refresh_token

'fd9fee11ceec11e7b2440242ac110003'

In [12]:
ff.getAll()

In [3]:
json_query = """{"tenantId":"e7dbd874-ea9d-4092-afa4-d2f68acaaaf4","id":1,"createdAt":"2015-11-16T17:34:37-02:00",
"subtotal":410.1,"interestFee":0,"payInFullDiscount":0,"interestRate":0,"quantityOfInstallments":1,
"shippingMethod":{"gatewayId":1,"id":6723,"name":"Frete Padrão","deliveryDeadLine":"5 dias","cost":1,"adjustedCost":1},
"total":411.1,"shippingAddress":{"id":1,"recipientName":"José da Silva","local":"José da Silva","phoneAreaCode":"41",
"phoneNumber":"3333-3333","phoneAreaCodeAlternative":null,
"streetNumber":"100"},
"package":{"id":null,"description":null,"value":0},"paymentMethods":[{"id":54,"name":"Faturado","installments":1,"value":411.1,"sequential":1,"paymentMethodGroup":{"id":25,"name":"Interno"},"paymentMethodCategory":{"id":13,"name":"Faturado"}}]}"""
json_sample = json.loads(json_query)
json_sample

{'createdAt': '2015-11-16T17:34:37-02:00',
 'id': 1,
 'interestFee': 0,
 'interestRate': 0,
 'package': {'description': None, 'id': None, 'value': 0},
 'payInFullDiscount': 0,
 'paymentMethods': [{'id': 54,
   'installments': 1,
   'name': 'Faturado',
   'paymentMethodCategory': {'id': 13, 'name': 'Faturado'},
   'paymentMethodGroup': {'id': 25, 'name': 'Interno'},
   'sequential': 1,
   'value': 411.1}],
 'quantityOfInstallments': 1,
 'shippingAddress': {'id': 1,
  'local': 'José da Silva',
  'phoneAreaCode': '41',
  'phoneAreaCodeAlternative': None,
  'phoneNumber': '3333-3333',
  'recipientName': 'José da Silva',
  'streetNumber': '100'},
 'shippingMethod': {'adjustedCost': 1,
  'cost': 1,
  'deliveryDeadLine': '5 dias',
  'gatewayId': 1,
  'id': 6723,
  'name': 'Frete Padrão'},
 'subtotal': 410.1,
 'tenantId': 'e7dbd874-ea9d-4092-afa4-d2f68acaaaf4',
 'total': 411.1}

In [4]:
ent = entityTemplateCarol.createTemplate(token_object)

ent.addFields(json_sample)

In [9]:
ent.all_possible_fields.keys()

NameError: name 'ent' is not defined

In [22]:
class entIntType(object):
    ent_type = 'long'


class entDoubleType(object):
    ent_type = 'double'


class entStringType(object):
    ent_type = "string"


class entNullType(object):
    ent_type = "string"


class entBooleanType(object):
    ent_type = "boolean"


class entArrayType(object):
    ent_type = "nested"


class entObjectType(object):
    ent_type = "object"


class entType(object):
    @classmethod
    def get_ent_type_for(cls, t):
        """docstring for get_schema_type_for"""
        SCHEMA_TYPES = {
            type(None): entNullType,
            str: entStringType,
            int: entIntType,
            float: entDoubleType,
            bool: entBooleanType,
            list: entArrayType,
            dict: entObjectType,
        }

        schema_type = SCHEMA_TYPES.get(t)

        if not schema_type:
            raise JsonEntTypeNotFound("There is no schema type for  %s.\n Try:\n %s" % (
            str(t), ",\n".join(["\t%s" % str(k) for k in SCHEMA_TYPES.keys()])))
        return schema_type


class JsonEntTypeNotFound(Exception):
    pass

In [23]:
ent_ =  ent.all_possible_fields.get('id',[]).copy()
ent_.pop('mdmCreated') 
ent_.pop('mdmLastUpdated') 
ent_.pop('mdmTenantId')
ent_

{'mdmAnalyzer': 'STANDARD',
 'mdmDescription': {},
 'mdmEntityType': 'mdmField',
 'mdmFieldType': 'PRIMITIVE',
 'mdmFields': [],
 'mdmFieldsFull': {},
 'mdmForeignKeyField': False,
 'mdmId': '1da76a80a85f11e7a2900242ac110003',
 'mdmIndex': 'ANALYZED',
 'mdmLabel': {'en-US': 'id'},
 'mdmMappingDataType': 'LONG',
 'mdmName': 'id',
 'mdmTags': []}

In [25]:
def create_field(mdmName,value,parentId = ''): 
    payload = {"mdmName":mdmName,"mdmMappingDataType":entity_type.ent_type,
           "mdmLabel":{"en-US": mdmName},"mdmDescription":{"en-US": mdmName}}
    entity_type = entType.get_ent_type_for(type(value))
    
    if entity_type == entObjectType and len(value) > 0:
        for key, val in value.items():
            payload['mdmFieldType'] = 'NESTED'
            print('criando NESTED')
            parentId = 1234
            create_field(key,val,parentId = parentId)
    
    elif entity_type == entArrayType and len(value) > 0:
        pass
    
    if entity_type.ent_type == 'nested':
        
        payload['mdmFieldType'] = 'NESTED'
        print('criando NESTED')
        parentId = 1234
        _, parentId = create_field()
    else:
        payload['mdmFieldType'] = 'PRIMITIVE'   
        print('criando PRIMITIVE')

    return payload, parentId

for prop, value in json_sample.items():
    entity_type = entType.get_ent_type_for(type(value))
    if prop in ent.all_possible_fields.keys():
        ent_ =  ent.all_possible_fields.get(prop,[]).copy()
        ent_.pop('mdmCreated') 
        ent_.pop('mdmLastUpdated') 
        ent_.pop('mdmTenantId') 
        if (ent_['mdmMappingDataType'].lower() == entity_type.ent_type) :
            print('SEND FIELD TO DM CREATED')
        else:
            raise('problem')
    else:
        to_create = create_field(prop,value)
        print(to_create)

criando PRIMITIVE
({'mdmName': 'tenantId', 'mdmMappingDataType': 'string', 'mdmLabel': {'en-US': 'tenantId'}, 'mdmDescription': {'en-US': 'tenantId'}, 'mdmFieldType': 'PRIMITIVE'}, '')
SEND FIELD TO DM CREATED
criando PRIMITIVE
({'mdmName': 'createdAt', 'mdmMappingDataType': 'string', 'mdmLabel': {'en-US': 'createdAt'}, 'mdmDescription': {'en-US': 'createdAt'}, 'mdmFieldType': 'PRIMITIVE'}, '')
criando PRIMITIVE
({'mdmName': 'subtotal', 'mdmMappingDataType': 'double', 'mdmLabel': {'en-US': 'subtotal'}, 'mdmDescription': {'en-US': 'subtotal'}, 'mdmFieldType': 'PRIMITIVE'}, '')
criando PRIMITIVE
({'mdmName': 'interestFee', 'mdmMappingDataType': 'long', 'mdmLabel': {'en-US': 'interestFee'}, 'mdmDescription': {'en-US': 'interestFee'}, 'mdmFieldType': 'PRIMITIVE'}, '')
criando PRIMITIVE
({'mdmName': 'payInFullDiscount', 'mdmMappingDataType': 'long', 'mdmLabel': {'en-US': 'payInFullDiscount'}, 'mdmDescription': {'en-US': 'payInFullDiscount'}, 'mdmFieldType': 'PRIMITIVE'}, '')
criando PRIMITI

TypeError: create_field() missing 3 required positional arguments: 'mdmName', 'value', and 'entity_type'